# Research Request - Service Data for SJCOG #1476

explore trying to answer these questions from the issue:

- share some information from NTD on what % of services are provided by SJRTD.
- see if NTD can help us tell what each operators need is for that region.
- see if NTD shows you the operations just in a particular UZA.
- Are you able to see specifically within the Tracy boundaries which percentage of service is provided by City of Tracy vs SJRTD?
- Can NTD tell you the funding for each operator by source?
- Can you tell what level of 5307, 5311, TDA, LCTOP etc they [operators?] get from there?

However, these question revolve around knowing the proportion of service that happens between area, not just the agencies main area. Example, the UZA for SJRTD is "Stockton, CA". But SJRTD runs service that goes outside of Stockton (some bus routes go to Tracy, Ripon, Lodi)

In [1]:
import altair as alt
import pandas as pd
from calitp_data_analysis.tables import tbls
from siuba import _, collect, count, filter, group_by, select, show_query, summarize

# from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [8]:
sj_cities = [
    "Stockton",
    "Escalon",
    "Tracy",
    "Lathrop",
    "Lodi",
    "Manteca",
    "Mountain House",
    "Ripon",
    "Acampo",
    "French Camp",
    "Garden Arcres",
]

In [34]:
ca = (
    tbls.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_vrh()
    >> filter(
        _.state == "CA",
        # _.city == "Stockton"
        # _.report_year == 2023
    )
    >> group_by(
        _.agency_name, 
        _.ntd_id, 
        _.reporter_type, 
        _.primary_uza_name, 
        _.last_report_year, 
        _.city
    )
    >> summarize(total_vrh=_.vrh.sum())
    >> collect()
)  # .fillna(0)

ca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   agency_name       269 non-null    object 
 1   ntd_id            254 non-null    object 
 2   reporter_type     269 non-null    object 
 3   primary_uza_name  269 non-null    object 
 4   last_report_year  269 non-null    int64  
 5   city              255 non-null    object 
 6   total_vrh         268 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 14.8+ KB


In [35]:
ca["city"].isna().value_counts()

False    255
True      14
Name: city, dtype: int64

In [36]:
ca.fillna("none")["city"].isna().value_counts()

False    269
Name: city, dtype: int64

In [37]:
# How many agencies have Tracy as their primary uza?
for i in sj_cities:
    display(
        f"Agencies with {i} as UZA",
        ca[ca["primary_uza_name"].str.contains(i)],
        f"Agencies with {i} as City",
        ca[ca["city"].notna() & ca["city"].str.contains(i)],
    )

'Agencies with Stockton as UZA'

,agency_name,ntd_id,reporter_type,primary_uza_name,last_report_year,city,total_vrh
7,Altamont Corridor Express (ACE),90182,Full Reporter,"Stockton, CA",2023,Stockton,529179.0
131,San Joaquin Regional Transit District (RTD),90012,Full Reporter,"Stockton, CA",2023,Stockton,7249405.0
254,San Joaquin Council (SJCOG),99422,Full Reporter,"Stockton, CA",2023,Stockton,650385.0


'Agencies with Stockton as City'

,agency_name,ntd_id,reporter_type,primary_uza_name,last_report_year,city,total_vrh
7,Altamont Corridor Express (ACE),90182,Full Reporter,"Stockton, CA",2023,Stockton,529179.0
131,San Joaquin Regional Transit District (RTD),90012,Full Reporter,"Stockton, CA",2023,Stockton,7249405.0
254,San Joaquin Council (SJCOG),99422,Full Reporter,"Stockton, CA",2023,Stockton,650385.0


'Agencies with Escalon as UZA'

...

,agency_name,ntd_id,reporter_type,primary_uza_name,last_report_year,city,total_vrh


'Agencies with Garden Arcres as UZA'

,agency_name,ntd_id,reporter_type,primary_uza_name,last_report_year,city,total_vrh


'Agencies with Garden Arcres as City'

,agency_name,ntd_id,reporter_type,primary_uza_name,last_report_year,city,total_vrh


In [38]:
# how many cities does SJRTD serve?
ca[ca["ntd_id"]=="90012"]

# just stockton?

,agency_name,ntd_id,reporter_type,primary_uza_name,last_report_year,city,total_vrh
131,San Joaquin Regional Transit District (RTD),90012,Full Reporter,"Stockton, CA",2023,Stockton,7249405.0


In [ ]:
# what are the routes for SJRTD?
# what GTFS table is there 
# mart_transit_database.bridge_organizations_x_services_managed

